In [7]:
import os
import uuid
import pandas as pd
from pathlib import Path
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
import whisper
from transformers import pipeline

# Load models
asr_model = whisper.load_model("base")
sentiment_pipeline = pipeline("sentiment-analysis")

INPUT_DIR = "input_files"
OUTPUT_DIR = "output_data"
CHUNK_MS = 5000

def extract_audio(video_path, audio_path):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

def split_audio(audio_path):
    audio = AudioSegment.from_file(audio_path)
    chunks = []
    for i in range(0, len(audio), CHUNK_MS):
        chunk = audio[i:i+CHUNK_MS]
        chunks.append((i / 1000.0, chunk))
    return chunks

def transcribe_chunk(chunk):
    temp = "temp_chunk.wav"
    chunk.export(temp, format="wav")
    result = asr_model.transcribe(temp)
    os.remove(temp)
    return result["text"]

def analyze_sentiment(text):
    result = sentiment_pipeline(text)[0]
    return result["label"]

def process_video(file_path):
    base = Path(file_path).stem
    audio_path = f"{base}.wav"
    extract_audio(file_path, audio_path)

    rows = []
    for start, chunk in split_audio(audio_path):
        text = transcribe_chunk(chunk)
        sentiment = analyze_sentiment(text)
        rows.append({
            "timestamp": round(start, 2),
            "transcription": text,
            "sentiment": sentiment
        })

    os.remove(audio_path)
    df = pd.DataFrame(rows)
    out_path = f"output_data/{base}_{uuid.uuid4().hex}.csv"
    df.to_csv(out_path, index=False)
    print(f" Output saved to: {out_path}")

# Run
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
for file in os.listdir(INPUT_DIR):
    if file.lower().endswith(('.mp4', '.mov')):
        process_video(os.path.join(INPUT_DIR, file))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


MoviePy - Writing audio in Experimenter_CREW_999_1_All_1731617801.wav


MoviePy - Done.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 Output saved to: output_data/Experimenter_CREW_999_1_All_1731617801_a9621a03c2434d9b97e1bae74397739b.csv
